In [2]:
import pandas as pd
df = pd.read_csv('/home/vamsi/Documents/loan_defaulter_challenge/train_indessa.csv')

In [6]:
df.columns.values

array(['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
       'batch_enrolled', 'int_rate', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code', 'addr_state',
       'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'application_type', 'verification_status_joint', 'last_week_pay',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'loan_status'], dtype=object)

In [8]:
def cleaning(df):
    df['open/total_acc'] = df['open_acc'] / df['total_acc']
    df['zip_addr_code'] = df['zip_code'] + df['addr_state']
    df['zip_addr_code'] = df['zip_addr_code'].str.replace(r'xx', '').astype('object')
    df_train = df[['member_id','loan_amnt','term','int_rate','grade','emp_length','home_ownership','annual_inc','verification_status','purpose','zip_addr_code','dti','delinq_2yrs','open/total_acc','pub_rec','revol_bal','revol_util','mths_since_last_major_derog','tot_coll_amt','tot_cur_bal','total_rev_hi_lim','loan_status']]
    return df_train
if __name__ == '__main__':
    df_train_noise = cleaning(df)

In [9]:
null_features = []
for each_column_name in list(df_train_noise.columns.values):
    if df_train_noise[each_column_name].isnull().values.any():
        null_features.append([each_column_name,df_train_noise[each_column_name].isnull().sum()])
null_features

[['annual_inc', 3],
 ['delinq_2yrs', 16],
 ['open/total_acc', 16],
 ['pub_rec', 16],
 ['revol_util', 287],
 ['mths_since_last_major_derog', 399448],
 ['tot_coll_amt', 42004],
 ['tot_cur_bal', 42004],
 ['total_rev_hi_lim', 42004]]

In [10]:
from scipy.stats import mode
def filling_null_values(df,null_features):
    for each_feature in range(0,len(null_features)-3):
        df[null_features[each_feature][0]].fillna(-1,inplace=True)
    for each_element in range(6,len(null_features)):
        df[null_features[each_element][0]].fillna(mode(df[null_features[each_element][0]]).mode[0], inplace=True)
    return df
if __name__ == '__main__':
    df_free_from_null = filling_null_values(df_train_noise,null_features)

/home/vamsi/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [11]:
null = []
for each_column_name in list(df_free_from_null.columns.values):
    if df_free_from_null[each_column_name].isnull().values.any():
        null.append([each_column_name,df_free_from_null[each_column_name].isnull().sum()])
null

[]

In [12]:
df_free_from_null.to_csv('train_data_categorical.csv',sep=',',index=False)


In [7]:
import numpy as np
categorical_variables_list = []
for each_variable in list(df_free_from_null.columns.values):
    if df_free_from_null[each_variable].dtype == np.object:
        categorical_variables_list.append(each_variable)
categorical_variables_list

['term',
 'grade',
 'emp_length',
 'home_ownership',
 'verification_status',
 'purpose',
 'zip_addr_code']

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for each_feature in categorical_variables_list:
    df_free_from_null[each_feature] = le.fit_transform(df_free_from_null[each_feature])
df_free_from_null.dtypes

/home/vamsi/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


member_id                        int64
loan_amnt                        int64
term                             int64
int_rate                       float64
grade                            int64
emp_length                       int64
home_ownership                   int64
annual_inc                     float64
verification_status              int64
purpose                          int64
zip_addr_code                    int64
dti                            float64
delinq_2yrs                    float64
open/total_acc                 float64
pub_rec                        float64
revol_bal                        int64
revol_util                     float64
mths_since_last_major_derog    float64
tot_coll_amt                   float64
tot_cur_bal                    float64
total_rev_hi_lim               float64
dtype: object

In [10]:
df_free_from_null.to_csv('test_data_clean.csv',sep=',',index=False)
